In [1]:
from __future__ import print_function
from pathlib import Path
from random import shuffle
import random

import argparse
import copy
import numpy as np
import sys
import tensorflow as tf
import scipy.sparse as sp
import sys

import sys
sys.path.append("..") 
from deeplp.models.deeplp_att import DeepLP_ATT
from deeplp.models.deeplp_edge import DeepLP_Edge
from deeplp.models.deeplp_wrbf import DeepLP_WRBF
from deeplp.models.lp import LP
from deeplp.utils import (calc_masks, create_seed_features, load_data,
                          num_layers_dict, prepare_data, random_unlabel)


/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
def acc(unlabeled_pred,unlabeled_indices,true_labels):
    y_pred = np.argmax(unlabeled_pred, axis=1)
    y_true = np.argmax(true_labels[unlabeled_indices], axis=1)
    accuracy = np.mean(y_pred == y_true)
    return accuracy

In [10]:
closed_accs = []
iter_accs = []
#     for i,data in enumerate(['flip_cora','flip_dblp','flip_flickr','flip_imdb','flip_industry','linqs_citeseer','linqs_cora','linqs_pubmed']):
for i,data in enumerate(['linqs_citeseer','linqs_cora']):
    print(data)
    if data[:4] == 'flip':
        model = 'att'
    else:
        model = 'edge'

    for seed in range(100):
        true_labels, edge_features, graph \
        = load_data(data,model=model)

        labeled_indices, unlabeled_indices = \
            random_unlabel(true_labels,0.99,
                           seed=seed)
        num_nodes, num_classes = true_labels.shape

        labels, is_labeled = calc_masks(true_labels, labeled_indices, unlabeled_indices)
        lp = LP()

        unlabeled_pred = lp.closed_sp(labels, graph, labeled_indices,
                                      unlabeled_indices)
        ac = acc(unlabeled_pred,unlabeled_indices,true_labels)
        closed_accs.append(ac)
        unlabeled_pred = lp.iter_sp(labels, graph, is_labeled,
                                    100, unlabeled_indices)
        ac = acc(unlabeled_pred,unlabeled_indices,true_labels)
        iter_accs.append(ac)        
            

linqs_citeseer
linqs_cora


In [13]:
np.savetxt('t_lp_val.csv',val_accuracies)


In [19]:
print(np.mean(closed_accs[:10]),np.mean(closed_accs[100:110]))
print(np.std(closed_accs[:10]),np.std(closed_accs[100:110]))

0.5007662835249043 0.4339430894308943
0.08225766769272637 0.08083182822192132


In [20]:
print(np.mean(iter_accs[:10]),np.mean(iter_accs[100:110]))
print(np.std(iter_accs[:10]),np.std(iter_accs[100:110]))

0.5414750957854405 0.5852439024390244
0.0728738376819388 0.07406478407494847


In [21]:
print(np.mean(closed_accs[:100]),np.mean(closed_accs[100:]))
print(np.std(closed_accs[:100]),np.std(closed_accs[100:]))

0.4975478927203066 0.4479186991869919
0.0719257229752979 0.10095589271121252


In [22]:
print(np.mean(iter_accs[:100]),np.mean(iter_accs[100:]))
print(np.std(iter_accs[:100]),np.std(iter_accs[100:]))

0.5572030651340997 0.5648333333333334
0.062226608793058094 0.08905980835710575


In [24]:
gcn = ["0.6085365","0.5882113","0.5975609","0.5999999","0.611382","0.59878045","0.6719511","0.57357717","0.62642264","0.5264226","0.6341462","0.66341454","0.6126015","0.62601614","0.64796734","0.47845525","0.44227636","0.48536575","0.67357713","0.5752032","0.57276416","0.51178855","0.6703251","0.59512186","0.6422763","0.48699176","0.5882113","0.5146341","0.65528446","0.41056904","0.6418698","0.6914633","0.58252025","0.65894294","0.46097556","0.65975595","0.63373977","0.5170731","0.6670731","0.57154465","0.5642276","0.5479674","0.5479674","0.58292675","0.6353658","0.6792681","0.6134146","0.6300812","0.5853658","0.6247966","0.65569097","0.65325195","0.5483739","0.59959334","0.62520313","0.64756083","0.5780487","0.613008","0.63821125","0.6739836","0.5414633","0.42439014","0.68983734","0.43902433","0.5983739","0.5609755","0.45853654","0.7126015","0.5792682","0.49227637","0.6817072","0.65406495","0.60487795","0.5963414","0.5195121","0.48008123","0.5971544","0.43983737","0.6451219","0.5821138","0.52682924","0.6528454","0.6516259","0.60650396","0.4288617","0.52154464","0.51178855","0.6288617","0.35934952","0.6060974","0.7508128","0.35894305","0.6325202","0.42276418","0.660569","0.6117885","0.39837393","0.55975604","0.6008129","0.6426828"]


In [26]:
gcn = [float(el) for el in gcn]

In [28]:
np.mean(gcn[:10])

0.600284456

In [29]:
np.mean(gcn)

0.5824999077

In [30]:
gcn_citeseer = ["0.56752926","0.5632189","0.46264365","0.5402302","0.5574717","0.52203083","0.56752926","0.5043105","0.63314277","0.28591955","0.62787455","0.4707854","0.20737547","0.5838129","0.68582517","0.541188","0.3429118","0.66331536","0.5081419","0.49808437","0.44061294","0.5445406","0.4990423","0.49425292","0.61446446","0.4358237","0.5086208","0.53879344","0.55699277","0.5081419","0.5316095","0.45881227","0.4865901","0.54262483","0.48323756","0.48515326","0.52011514","0.6752887","0.6149434","0.56705034","0.55459815","0.45593867","0.5320884","0.5249045","0.5919547","0.65373677","0.61015403","0.5795025","0.4851532","0.3371647","0.50766295","0.5081419","0.5780657","0.49185836","0.527778","0.68007797","0.5957862","0.44061297","0.5306516","0.59195477","0.54837203","0.5545981","0.39942527","0.55172455","0.5019159","0.5795025","0.569445","0.4856322","0.4482758","0.55986637","0.5201151","0.5153258","0.43630263","0.4425287","0.6101541","0.54837203","0.53927237","0.60632265","0.6542157","0.49281612","0.5311306","0.4530651","0.54693526","0.6024912","0.5186783","0.4257662","0.51388913","0.41427198","0.33141756","0.6293113","0.66331536","0.5866865","0.44588116","0.3721264","0.35057467","0.57231855","0.5526824","0.63026917","0.5943494","0.7107296"
]

In [31]:
gcn_citeseer = [float(el) for el in gcn_citeseer]

In [32]:
np.mean(gcn_citeseer[:10]), np.mean(gcn_citeseer)

(0.5204026620000001, 0.5262791127)